In [1]:
import os
import numpy as np
import time

from monai.handlers import (
    MeanDice,
    from_engine,
)
from config import get_config
from monai.utils import first

In [2]:
config = "/data/kehyeong/project/MONAI_examples/dynunet_pipeline/config_evalImg_toy_220317.yaml"
# checkpoint = "/data/train/running/l/model_roi_try1_220217/models/net_key_metric=0.3331.pt"
eval_dataset = "/work/NeuroI-models/ke-monai/data/roi/dataset_test_roi_toy2.csv"
eval_output_dir = "./runs_eval2"

config = get_config(config)
data_dir = config["data_dir"]
# image_file_path = config["image_file_path"]
# label_file_path = config["label_file_path"]
# mask_file_path = config["mask_file_path"]
pred_file_path = config["pred_file_path"]
label_file_path = config["label_file_path"]

eval_batch_size = config["eval"]["batch_size"] 
eval_num_workers = config["eval"]["num_workers"]
num_classes = config["num_classes"]
# patch_size = config["patch_size"]

spacing = [1.0, 1.0, 1.0]

In [3]:
# f = open(eval_dataset)
# rows = csv.DictReader(f)
# data = []
# for row in rows:
# #     id = row["id"]
# #     for aid in eval_aids:
#     print(row)

In [4]:
from monai.transforms import (
    AddChanneld,
    CastToTyped,
    Compose,
    LoadImaged,
    Orientationd,
    EnsureTyped,
    MapLabelValued,
    AsDiscreted
)



from monai.transforms.compose import MapTransform

orig_label_classes, target_label_classes = (                                      # 109개 ROIs from zag
    np.array([   0,    2,    3,    4,    5,    7,    8,   10,   11,   12,   13,
         14,   15,   16,   17,   18,   24,   26,   28,   30,   31,   41,
         42,   43,   44,   46,   47,   49,   50,   51,   52,   53,   54,
         58,   60,   62,   63,   77,   80,   85,  251,  252,  253,  254,
        255, 1000, 1002, 1003, 1005, 1006, 1007, 1008, 1009, 1010, 1011,
       1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022,
       1023, 1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1034, 1035,
       2000, 2002, 2003, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
       2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023,
       2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2034, 2035]),
    np.array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108])
    )

def get_eval_from_img_transform():
    keys = ["pred", "label"]
    transforms = [
        LoadImaged(keys=keys),
        AddChanneld(keys=keys),
        Orientationd(keys=keys, axcodes="RAS"),
        MapLabelValued(
            keys=keys, 
            orig_labels=orig_label_classes, 
            target_labels=target_label_classes,
            dtype=np.uint8
        ),
#         Lambdad(keys='image', func=lambda x: np.where(x==-0, 0, x)),
        # ScaleIntensityd(keys=["image"], minv=0.0, maxv=1.0),
        # ScaleIntensityRanged(
        #     keys=["image"],
        #     a_min=-175,
        #     a_max=250,
        #     b_min=0.0,
        #     b_max=1.0,
        #     clip=True,
        # ),
#         ConcatItemsd(keys=["image", "mask"], name="image"),
        # SelectItemsd(
        #     keys=["image", "label", 
        #           "image_meta_dict", "label_meta_dict", 
        #           "image_transforms", "label_transforms"]
        # ),
        Checkpixdim(
            keys=keys,
            pixdim=spacing,
        ),        
#         PreprocessAnisotropic(
#             keys=["image", "label"],
#             clip_values=clip_values,
#             pixdim=spacing,
#             normalize_values=normalize_values,
#             model_mode="validation",
#         ),
        CastToTyped(keys=keys, dtype=(np.uint8, np.uint8)),
        EnsureTyped(keys=keys),
#         AsDiscreted(keys=keys, to_onehot=num_classes)
        # DeleteItemsd(
        #     keys=["mask", "mask_meta_dict", "mask_transforms"]
        # ),
    ]
    return Compose(transforms)

class Checkpixdim(MapTransform):
    # 주어진 spacing과 업로드한 이미지들의 pixel dim이 일치 하는지 확인 by K.E
    def __init__(
        self,
        keys,
        pixdim,
    ) -> None:
        super().__init__(keys)
        self.keys = keys
        self.target_spacing = pixdim
    def __call__(self, data):
        # load data
        d = dict(data)
#         image_spacings = d["pred_meta_dict"]["pixdim"][1:4].tolist()
        pred_spacings = d["pred_meta_dict"]["pixdim"][1:4].tolist()
        label_spacings = d["label_meta_dict"]["pixdim"][1:4].tolist()
        
        if self.target_spacing != pred_spacings:
            raise AssertionError(f"pred nifti img의 pixdim({pred_spacings})이 지정된 spacing({self.target_spacing})과 일치 하지 않습니다.")
        elif self.target_spacing != pred_spacings:
            raise AssertionError(f"label nifti img의 pixdim({label_spacings})이 지정된 spacing({self.target_spacing})과 일치 하지 않습니다.")
            
        return d

In [5]:
### dataset_roi에 심어야 함
import csv
from monai.data import (
    CacheDataset,
    DataLoader,
    partition_dataset,
)

eval_aids = [
    "00_0",
    # "01_d",
]

def get_eval_data_from_img(
    id_file, data_dir, 
    pred_file_pattern, 
    label_file_pattern
):
    f = open(id_file)
    rows = csv.DictReader(f)
    data = []
    for row in rows:
        id = row["id"]
        for aid in eval_aids:
            data.append(
                {
                    "pred": pred_file_pattern.format(dir=data_dir, id=id, aid=aid),
                    "label": label_file_pattern.format(dir=data_dir, id=id, aid=aid),
                }
            )
    return data

def get_eval_loader(
    data_dir,
    id_file,
    pred_file_pattern,
    label_file_pattern,
    batch_size,
    num_workers=1,
):
    data = get_eval_data_from_img(
        id_file, data_dir, 
        pred_file_pattern, label_file_pattern
    )

    transform = get_eval_from_img_transform()

#     if torch.cuda.is_available() and dist.get_world_size() > 1:
#         data = partition_dataset(
#             data=data,
#             shuffle=False,
#             num_partitions=dist.get_world_size(),
#             even_divisible=False,
#         )[dist.get_rank()]

    dataset = CacheDataset(data=data, transform=transform, num_workers=4)   # 4

    data_loader = DataLoader(
        dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers
    )

    return data_loader

In [6]:
if not os.path.exists(eval_output_dir):
    os.makedirs(eval_output_dir)
    
# if multi_gpu_flag:
#     dist.init_process_group(backend="nccl", init_method="env://")
#     device = torch.device(f"cuda:{local_rank}")
#     torch.cuda.set_device(device)
# else:
#     device = torch.device("cuda:6")
# #     device = torch.device("cpu")

eval_loader = get_eval_loader(
    data_dir=data_dir,
    id_file=eval_dataset,
    pred_file_pattern=pred_file_path,
    label_file_pattern=label_file_path,
    batch_size=eval_batch_size,
    num_workers=eval_num_workers,
#     multi_gpu_flag=multi_gpu_flag
)

Loading dataset: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.22s/it]


In [7]:
test_data = first(eval_loader)
print(test_data.keys())
print("pred shape:", test_data['pred'].shape)
print("pred dtype:", test_data['pred'].dtype)
print("label shape:", test_data['label'].shape)
print("label dtype:", test_data['label'].dtype)

dict_keys(['pred', 'label', 'pred_meta_dict', 'label_meta_dict', 'pred_transforms', 'label_transforms'])
pred shape: torch.Size([1, 1, 186, 230, 230])
pred dtype: torch.uint8
label shape: torch.Size([1, 1, 186, 230, 230])
label dtype: torch.uint8


----

### Test.

In [8]:
np.unique(test_data['pred']) 

array([  0,   1,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  20,  21,  23,  24,  25,  26,  27,  28,
        29,  30,  31,  32,  33,  34,  36,  37,  39,  40,  41,  42,  43,
        44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,
        57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
        70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,
        83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,
        96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108],
      dtype=uint8)

In [9]:
np.unique(test_data['label'])

array([  0,   1,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  20,  21,  23,  24,  25,  26,  27,  28,
        29,  30,  31,  32,  33,  34,  36,  37,  39,  40,  41,  42,  43,
        44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,
        57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
        70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,
        83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,
        96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108],
      dtype=uint8)

In [10]:
test_data['pred_meta_dict']

{'sizeof_hdr': tensor([348], dtype=torch.int32),
 'extents': tensor([0], dtype=torch.int32),
 'session_error': tensor([0], dtype=torch.int16),
 'dim_info': tensor([0], dtype=torch.uint8),
 'dim': tensor([[  3, 186, 230, 230,   1,   1,   1,   1]], dtype=torch.int16),
 'intent_p1': tensor([0.]),
 'intent_p2': tensor([0.]),
 'intent_p3': tensor([0.]),
 'intent_code': tensor([0], dtype=torch.int16),
 'datatype': tensor([16], dtype=torch.int16),
 'bitpix': tensor([32], dtype=torch.int16),
 'slice_start': tensor([0], dtype=torch.int16),
 'pixdim': tensor([[1., 1., 1., 1., 0., 0., 0., 0.]]),
 'vox_offset': tensor([0.]),
 'scl_slope': tensor([nan]),
 'scl_inter': tensor([nan]),
 'slice_end': tensor([0], dtype=torch.int16),
 'slice_code': tensor([0], dtype=torch.uint8),
 'xyzt_units': tensor([2], dtype=torch.uint8),
 'cal_max': tensor([0.]),
 'cal_min': tensor([0.]),
 'slice_duration': tensor([0.]),
 'toffset': tensor([0.]),
 'glmax': tensor([0], dtype=torch.int32),
 'glmin': tensor([0], dtype=

In [11]:
pred_spacings = test_data['pred_meta_dict']["pixdim"][0][1:4].tolist()
pred_spacings

[1.0, 1.0, 1.0]

In [12]:
test_data['pred_meta_dict']["pixdim"]

tensor([[1., 1., 1., 1., 0., 0., 0., 0.]])

---------------

## Evaluation 

In [15]:
from monai.transforms import AsDiscrete, Transform
from monai.data import decollate_batch

post_transforms = AsDiscreted(keys=["pred", "label"], to_onehot=num_classes)

# def get_post_transforms():
#     keys = ["pred", "label"]
#     transforms = [
#         AsDiscreted(keys=keys, to_onehot=num_classes)
#     ]
#     return Compose(transforms)



In [ ]:
test_data

In [ ]:
decollate_batch(test_data)

In [ ]:
test_data = [post_transforms(i) for i in decollate_batch(test_data)]

In [ ]:
len(test_data)

In [ ]:
test_data[0]['pred'].shape

In [ ]:
## 각 데이터 모양
# test_data [{'pred': tensor, 'label': tensor}]
# val_output [tensor], val_labels [tensor]
val_outputs, val_labels = from_engine(["pred", "label"])(test_data)

In [ ]:
# a, b = test_data  ## Error!!

In [ ]:
test_data

In [ ]:
val_outputs

### 실험 1. dice_metric은 실행할 수록 쌓이는 구조인가?
* 쌓이는 구조는 아니고 한번 실행마다 넣어준 pred, label의 dice 결과 텐서를 출력해줌.
* 그런데 MONAI doc을 보니 `CumulativeIterationMetric`의 개념으로 metric는 aggregate final result를 출력가능하다고 나옴.
  * 방식 : `dice_metric.aggregate().item()`
  * 주의할 점: aggregate를 한번 더 하면 예상치 못한 쓰레기 결과도 함께 포함될 수 있음. 딱 처음 한번만 해야함.

In [ ]:
from monai.metrics import DiceMetric, compute_meandice
dice_metric = DiceMetric(include_background=False, reduction="mean")
result = dice_metric(y_pred=val_outputs, y=val_labels)
result

In [ ]:
result.numpy()

In [ ]:
# 차곡차곡 쌓이는 구조인가?
result = dice_metric(y_pred=val_outputs, y=val_labels)
result

In [ ]:
result.shape

In [ ]:
c = dice_metric.aggregate().item()
c

In [ ]:
dice_metric.reset()

### 실험 2. dice_metric에 리스트 텐서-[tensor] 구조가 아닌 그냥 tensor만 넣어도 계산될까?
-> 이상한 형태의 결과값이 나옴.. 결과 의도를 파악 못하겠음.. 리스트 구조로 꼭 넣는걸로..

In [ ]:
# [tensor] 구조가 아닌 그냥 tensor만 넣어도 계산될까? -> 계산 된다.
dice_metric = DiceMetric(include_background=False, reduction="mean")
a = dice_metric(y_pred=val_outputs[0], y=val_labels[0])
a.shape, a

### 실험 3. 한셀 코딩
* loader의 결과가 2차원 리스트 형식으로 들어가도록 해보자.

In [40]:
import pandas as pd

from monai.transforms import AsDiscreted, Transform
from monai.data import decollate_batch
from monai.metrics import DiceMetric

filename_list = []
total_sample_dice = []
post_transforms = AsDiscreted(keys=["pred", "label"], to_onehot=num_classes)

for idx, eval_data in enumerate(eval_loader):
    filename = eval_data['pred_meta_dict']['filename_or_obj'][0].split("/")[-1].split(".")[0]
    filename_list.append(filename)
    
    
    sample_num = idx+1
    eval_data = [post_transforms(i) for i in decollate_batch(eval_data)]
    eval_outputs, eval_labels = from_engine(["pred", "label"])(eval_data)
    
    dice_metric = DiceMetric(include_background=False, reduction="mean")
    sample_dice_result = dice_metric(y_pred=eval_outputs, y=eval_labels)
    sample_dice_result = sample_dice_result.numpy()
    total_sample_dice.append(sample_dice_result)
    
    print(f'sample : {filename}....... done')

# Average Mean Dice
avg_mean_dice = dice_metric.aggregate().item()
dice_metric.reset()

# arrange each sample's dice coefficient per ROIs
sample_num = sample_num * eval_batch_size
total_sample_dice = np.array(total_sample_dice)
total_sample_dice = total_sample_dice.reshape(sample_num, -1)

# add ROIs Means & total Mean
df = pd.DataFrame(total_sample_dice).T
df = pd.concat([pd.DataFrame(np.zeros(df.shape[1])).T, df])
df.columns = filename_list
df['roi_meanDice'] = df.mean(axis=1)
df['total_meanDice'] = avg_mean_dice
df.reset_index(inplace=True, drop=True)
df = df.applymap(lambda x: np.nan if x==0 else x)    # missing -> NaN 처리

# add orig_index, target_index
df_index = pd.DataFrame([orig_label_classes, target_label_classes]).T
df_index.columns = ['original_index', 'target_index']
df_mid = pd.concat([df_index, df], axis=1)

# mapping original name
mapping_f = '/data/kehyeong/project/NeuroI-models/ROI-pipeline/data/ROI_mapping_final_220314.txt'
df_name_map = pd.read_csv(mapping_f, sep='\t')
df_final = pd.merge(df_name_map, df_mid, how='right', left_on="label_label-from", right_on="original_index")
df_final.drop(columns=["label_label-from", "label_label-to"], inplace=True)

df_final

sample : s_SU0303_00_0_l....... done
sample : s_SU0197_00_0_l....... done


,ori_name,original_index,target_index,s_SU0303_00_0_l,s_SU0197_00_0_l,roi_meanDice,total_meanDice
0,Clear Label,0,0,NaN,NaN,NaN,1.0
1,L Cerebral-White-Matter,2,1,1.0,1.0,1.0,1.0
2,L Cerebral-Cortex,3,2,NaN,NaN,NaN,1.0
3,L Lateral-Ventricle,4,3,1.0,1.0,1.0,1.0
4,L Inf-Lat-Vent,5,4,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...
104,R superiorparietal,2029,104,1.0,1.0,1.0,1.0
105,R superiortemporal,2030,105,1.0,1.0,1.0,1.0
106,R supramarginal,2031,106,1.0,1.0,1.0,1.0
107,R transversetemporal,2034,107,1.0,1.0,1.0,1.0


In [41]:
df

,s_SU0303_00_0_l,s_SU0197_00_0_l,roi_meanDice,total_meanDice
0,NaN,NaN,NaN,1.0
1,1.0,1.0,1.0,1.0
2,NaN,NaN,NaN,1.0
3,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0
...,...,...,...,...
104,1.0,1.0,1.0,1.0
105,1.0,1.0,1.0,1.0
106,1.0,1.0,1.0,1.0
107,1.0,1.0,1.0,1.0


In [42]:
df['total_meanDice'] = df['roi_meanDice'].mean()

1.0

In [39]:
# 행/열 전환된 결과 파일 출력
df_transpose = df_final.T
new_header = df_transpose.iloc[0,:]
df_tf = df_transpose[1:]
df_tf.columns = new_header
df_tf

ori_name,Clear Label,L Cerebral-White-Matter,L Cerebral-Cortex,L Lateral-Ventricle,L Inf-Lat-Vent,L Cerebellum-White-Matter,L Cerebellum-Cortex,L Thalamus-Proper,L Caudate,L Putamen,...,R precentral,R precuneus,R rostralanteriorcingulate,R rostralmiddlefrontal,R superiorfrontal,R superiorparietal,R superiortemporal,R supramarginal,R transversetemporal,R insula
original_index,0,2,3,4,5,7,8,10,11,12,...,2024,2025,2026,2027,2028,2029,2030,2031,2034,2035
target_index,0,1,2,3,4,5,6,7,8,9,...,99,100,101,102,103,104,105,106,107,108
s_SU0303_00_0_l,NaN,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
s_SU0197_00_0_l,NaN,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
roi_meanDice,NaN,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
total_meanDice,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
